#Rolling 21 Day Sample standard Deviation

In [2]:
#importing all the important libraries
from pandas.io.data import DataReader
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import numpy as np


In [7]:
# Choose a time period 
d1 = datetime.datetime(2001, 01, 01)
d2 = datetime.datetime(2012, 01, 01)

#get the tickers
price = DataReader("^FTSE", "yahoo",d1,d2)['Adj Close']
price = price.asfreq('B').fillna(method='pad')

returns = price.pct_change()

### Rolling 21 day Standard Deviation
A rolling analysis of a time series model is often used to assess the model’s
stability over time. When analyzing financial time series data using a statistical
model, a key assumption is that the parameters of the model are
constant over time. However, the economic environment often changes considerably,
and it may not be reasonable to assume that a model’s parameters
are constant. A common technique to assess the constancy of a model’s
parameters is to compute parameter estimates over a rolling window of a
fixed size through the sample. If the parameters are truly constant over the
entire sample, then the estimates over the rolling windows should not be
too di􀀎erent. If the parameters change at some point during the sample,
then the rolling estimates should capture this instability.

In [9]:
pd.rolling_std(returns, 21).fillna(0)

2001-01-01    0
2001-01-02    0
2001-01-03    0
2001-01-04    0
2001-01-05    0
2001-01-08    0
2001-01-09    0
2001-01-10    0
2001-01-11    0
2001-01-12    0
2001-01-15    0
2001-01-16    0
2001-01-17    0
2001-01-18    0
2001-01-19    0
...
2011-12-12    0.013832
2011-12-13    0.014059
2011-12-14    0.014879
2011-12-15    0.014970
2011-12-16    0.014593
2011-12-19    0.014411
2011-12-20    0.013231
2011-12-21    0.013288
2011-12-22    0.013045
2011-12-23    0.013078
2011-12-26    0.013071
2011-12-27    0.011701
2011-12-28    0.011699
2011-12-29    0.009753
2011-12-30    0.009720
Freq: B, Length: 2870

In [ ]:


#choose the quantile
quantile=0.05
#the vol window
volwindow=50
#and the Var window for rolling 
varwindow=250

#simple VaR using all the data
unnormedquantile=pd.expanding_quantile(ret,quantile)

#similar one using a rolling window 
unnormedquantileR=pd.rolling_quantile(ret,varwindow,quantile)

#we can also normalize the returns by the vol
vol=pd.rolling_std(ret,volwindow)*np.sqrt(256)
unitvol=ret/vol

#and get the expanding or rolling quantiles
Var=pd.expanding_quantile(unitvol,quantile)
VarR=pd.rolling_quantile(unitvol,varwindow,quantile)

normedquantile=Var*vol
normedquantileR=VarR*vol

ret2=ret.shift(-1)

courbe=pd.DataFrame({'returns':ret2,
              'quantiles':unnormedquantile,
              'Rolling quantiles':unnormedquantileR,
              'Normed quantiles':normedquantile,
              'Rolling Normed quantiles':normedquantileR,
              })
courbe.plot()
plt.show()

courbe['nqBreak']=np.sign(ret2-normedquantile)/(-2) +0.5
courbe['nqBreakR']=np.sign(ret2-normedquantileR)/(-2) +0.5
courbe['UnqBreak']=np.sign(ret2-unnormedquantile)/(-2) +0.5
courbe['UnqBreakR']=np.sign(ret2-unnormedquantileR)/(-2) +0.5


nbdays=price.count()

print 'Number of returns worse than the VaR'
print 'Ideal Var                : ', (quantile)*nbdays
print 'Simple VaR               : ', np.sum(courbe['UnqBreak'])
print 'Normalized VaR           : ', np.sum(courbe['nqBreak'])
print '---------------------------'
print 'Ideal Rolling Var        : ', (quantile)*(nbdays-varwindow)
print 'Rolling VaR              : ', np.sum(courbe['UnqBreakR'])
print 'Rolling Normalized VaR   : ', np.sum(courbe['nqBreakR'])